In [1]:
import re

# 读取文本

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    text = f.read()
print(len(text))
print(text[:90])
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so i
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


# 创建词表

In [3]:
words = sorted(set(preprocessed))
vocab_size = len(words)
print(vocab_size)
vocab = {token:i for i, token in enumerate(words)}

1130


# 文本分词器

## 分词器

In [4]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [5]:
tokenizer = SimpleTokenizerV1(vocab)
text_test = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids_test = tokenizer.encode(text_test)
print(ids_test)
print(tokenizer.decode(ids_test))

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## 特殊词元

In [6]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:i for i, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


## 特殊词元分词器

In [7]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed] 
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

#### 测试

In [8]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text_test = " <|endoftext|> ".join((text1, text2))#用句子分隔符链接两个句子

print(text_test) #跟第一个一样,但不会报错了

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [9]:
tokenizer.encode(text_test)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [10]:
tokenizer.decode(tokenizer.encode(text_test))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## BPE分词

In [11]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))#验证下载并输出版本信息

tiktoken version: 0.9.0


In [12]:
tokenizer = tiktoken.get_encoding("gpt2")

In [13]:
text_test = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.")
integers = tokenizer.encode(text_test, allowed_special={"<|endoftext|>"})#输出分词的id,可以允许endoftext
print(integers)
strings = tokenizer.decode(integers)
#按照数字解码回去
print(strings)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


## 批处理数据集

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

In [15]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [16]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [18]:
dataloader = create_dataloader_v1(#raw_text 中创建一个数据加载器 但是所批次
    text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)#数据加载器 dataloader 转换为一个迭代器
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [19]:
dataloader = create_dataloader_v1(text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## Embedding

### 词元

In [20]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

# 测试
max_length = 4
dataloader = create_dataloader_v1(
    text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)
print(inputs.shape)

In [24]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


### 位置

In [26]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)
print(pos_embeddings)

torch.Size([4, 256])
tensor([[-0.3478, -1.2842, -1.3192,  ..., -0.4643,  0.8892, -0.4299],
        [-1.0796, -0.8647,  0.4097,  ..., -0.9574, -1.0857, -0.6222],
        [-2.2889, -0.0481, -0.5568,  ...,  0.3034, -1.3720, -1.3908],
        [-1.3982, -1.0234, -0.9757,  ..., -0.5593, -0.0705, -1.1475]],
       grad_fn=<EmbeddingBackward0>)


In [28]:
### 总输出

In [27]:
input_embeddings = token_embeddings + pos_embeddings#特征是词语信息跟位置信息的结合
print(input_embeddings.shape)

torch.Size([8, 4, 256])
